# April 10th, Wednesday
## Practical Session IV - Part 2


## 0. Setup the environment and load your data

In [ ]:
# Install scanpy and loompy if you don't have them already or if you are running on colab
! pip install scanpy loompy > _

In [ ]:
# Load the libraries we will use
import numpy as np
import pandas as pd
import scanpy as sc
import loompy
import matplotlib.pyplot as plt

In [ ]:
# Adjust the output for the figures
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=100, facecolor='white')
plt.rcParams['figure.figsize'] = (6, 6)
plt.rcParams['font.size'] = 16
sc.logging.print_header()

In [ ]:
# IF you DID NOT generate the normalize data, you can download it from the repository:
#!wget https://figshare.com/ndownloader/files/44904202 -O Data1_Norm.h5ad

In [ ]:
adata = sc.read_h5ad(#YOUR CODE HERE#) # enter loom/h5ad file name here
#adata = sc.read_loom(#YOUR CODE HERE#, var_names='var_names', obs_names='obs_names') # if you load from a loom file, specify the name for the indices of var and obs

In [ ]:
# Some scanpy versions might be asking for this
adata.uns['log1p']['base']=None

## 3. Feature selection

Now, we begin with dimensionality reduction, i.e. reducing the number of variables in the data by removing features (genes) with little variability among the cells and by combining highly similar features. This is important because normally you start with tens of thousands of genes and it is difficult to represent their patterns in a two-dimensional visualization.

One useful (but optional) step is to perform cell cycle characterization, as the cell cycle signature is often a strong convoluting factor with cell type signatures.

### **Exercise 1**:

Perform cell cycle characterization on your dataset. Plot a scatter plot of the S_score and G2M_score metadata created and stored in your AnnData object. Color the points by the assigned cell cycle phase. What percentage of your cells are in a proliferative state (S or G2M phases)?

In [ ]:
# your code here
S_genes_mouse = ['Mcm5', 'Pcna', 'Tyms', 'Fen1', 'Mcm2', 'Mcm4', 'Rrm1', 'Ung',
       'Gins2', 'Mcm6', 'Cdca7', 'Dtl', 'Prim1', 'Uhrf1', 'Cenpu',
       'Hells', 'Rfc2', 'Rpa2', 'Nasp', 'Rad51ap1', 'Gmnn', 'Wdr76',
       'Slbp', 'Ccne2', 'Ubr7', 'Pold3', 'Msh2', 'Atad2', 'Rad51', 'Rrm2',
       'Cdc45', 'Cdc6', 'Exo1', 'Tipin', 'Dscc1', 'Blm', 'Casp8ap2',
       'Usp1', 'Clspn', 'Pola1', 'Chaf1b', 'Brip1', 'E2f8']
G2M_genes_mouse = ['Hmgb2', 'Cdk1', 'Nusap1', 'Ube2c', 'Birc5', 'Tpx2', 'Top2a',
       'Ndc80', 'Cks2', 'Nuf2', 'Cks1b', 'Mki67', 'Tmpo', 'Cenpf',
       'Tacc3', 'Pimreg', 'Smc4', 'Ccnb2', 'Ckap2l', 'Ckap2', 'Aurkb',
       'Bub1', 'Kif11', 'Anp32e', 'Tubb4b', 'Gtse1', 'Kif20b', 'Hjurp',
       'Cdca3', 'Jpt1', 'Cdc20', 'Ttk', 'Cdc25c', 'Kif2c', 'Rangap1',
       'Ncapd2', 'Dlgap5', 'Cdca2', 'Cdca8', 'Ect2', 'Kif23', 'Hmmr',
       'Aurka', 'Psrc1', 'Anln', 'Lbr', 'Ckap5', 'Cenpe', 'Ctcf', 'Nek2',
       'G2e3', 'Gas2l3', 'Cbx5', 'Cenpa']

sc.tl.score_genes_cell_cycle(adata, s_genes=S_genes_mouse, g2m_genes=G2M_genes_mouse)

In [ ]:
# See which new variables were created
adata.obs.head(3)

In [ ]:
n2c = {"G1":"red", "S":"green", "G2M":"blue"} # use to assign each cell a color based on phase in the scatter plot
adata.obs['phase_color']=adata.obs.phase.map(#YOUR CODE HERE#) # use the map function to apply the n2c dictionary
adata.obs.head(3)

In [ ]:
# your code here
# Plot the G2M and S score of each cell, color them by phase assigned by scanpy (use the color column we just created)
plt.scatter(x=#YOUR CODE HERE#, y=#YOUR CODE HERE#, c=#YOUR CODE HERE#) # your code here!
plt.show()

In [ ]:
# How many cells do we have assigned to each cell cycle phase?
adata.obs["phase"].value_counts()

### **Exercise 2**:
Next, we want to select a subset (~1000-4000) of genes that are highly variable in the dataset. Genes that are highly or lowly expressed in all cells will not assist with discriminating different cell types from each other.

In [ ]:
# suggestion: start with 2000 highly variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=2000, min_disp=0.5,
                            max_disp=np.inf, min_mean=0.0125, max_mean=3, span=0.3, flavor='cell_ranger')

In [ ]:
adata

In [ ]:
# You could filter your data to keep ony the highly variable genes
# adata = adata[:, adata.var["highly_variable"]].copy() # actually do the filtering

In [ ]:
adata

### **Exercise 3:**

While correcting for technical covariates may be crucial to uncovering the underlying biological signal, correction for biological covariates serves to single out particular biological signals of interest. The most common biological data correction is to remove the effects of the cell cycle on the transcriptome, the number of raw counts that existed per cell, or the percentage of mitochondrial reads present. This data correction can be performed by a simple linear regression against a cell cycle score as implemented in scanpy.

In [ ]:
# your code here
sc.pp.regress_out(adata, 'phase') # specify which feature from adata.obs you want to regress out, if any

## 4. Principal component analysis

Dimensionality reduction methods seek to take a large set of variables and return a smaller set of components that still contain most of the information in the original dataset. One of the simplest forms of dimensionality reduction is PCA. Principal component analysis (PCA) is a mathematical procedure that transforms a number of possibly correlated (e.g., expression of genes in a network) variables into a (smaller) number of uncorrelated variables called principal components ("PCs").

In [ ]:
# Do PCA
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
# Results are stored in the .uns layer of the anndata
adata

### **Exercise 4**:
Visualize the first PCs, color by cell cycle phase and n_counts (unnormalized)


In [ ]:
# your code here
sc.pl.pca(adata, color=#YOUR CODE HERE#) # specify the phase, n_counts as colors based on the adata.obs metadata naming scheme

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True) # see contribution of each PC to variance

## 5. Dimensionality reduction

Compute the neighborhood graph of cells using the PCA representation of the data matrix. The purpose of this step is to understand the “distance” between individual cells in the lower-dimensional PCA space, important for creating 2D scatter plot representations of your data. The number of neighbors used will influence how much the data is smoothened, which is a necessary step due to the sparsity (missing values) widely present in scRNA-seq data compared to bulk methods.


In [ ]:
sc.pp.neighbors(adata, n_neighbors=#YOUR CODE HERE#, n_pcs=#YOUR CODE HERE#) # specify the number of neighbors and number of PCs you wish to use

Embedding the graph in a 2D representation can be performed using either tSNE or UMAP algorithms.

### **Exercise 5**:

Run tSNE and UMAP algorithms on your data after completing the previous steps with default parameters. We will evaluate the quality of each approach in later exercises.


In [ ]:
sc.tl.tsne(adata, random_state=42)

In [ ]:
sc.tl.umap(adata, alpha=0.3, min_dist=0.5, random_state=42)

In [ ]:
# visualize the tSNE colored by phase
sc.pl.tsne(#YOUR CODE HERE#, use_raw=False, color=#YOUR CODE HERE#, wspace=0.3, ncols=3)

In [ ]:
# visualize the UMAP colored by phase
sc.pl.umap(#YOUR CODE HERE#, use_raw=False, color=#YOUR CODE HERE#, wspace=0.3, ncols=3)

### **Exercise 6**:

Now save your data again.

If you use .loom format, this time you *must* use the option *write_obsm_varm=True* to store .obsm and .varm layers in the loom file.

Note that the .uns layer will not be saved in .loom, so you will have to recalculate the neighbors later on!

In [ ]:
adata

In [ ]:
# Your code here
adata.write_h5ad(#YOUR CODE HERE#) # Choose a filename to store your data

In [ ]:
# Your code here
#adata.write_loom('Data1_DimRed.loom', write_obsm_varm=True) # Choose a filename to store your data